# Pyspark Usage with Delta Lake & Minio

This notebook shows how to write a CSV file directly to Minio, and also how to write and read a managed Delta Lake table in Minio.

Click the Table of Contents button in the left JupyterLab sidebar (the button on the far left of this browser window that looks like a bulleted list) to see the types of examples provided. **Make sure to run all the cells above a given section, since most examples in this notebook depend on those above them**

## Get Environment Variables for Minio (S3) Connection

In [1]:
import pyspark
import os

In [2]:
os.environ 
## Should see S3_ENDPOINT, S3_ACCESS_KEY, and S3_SECRET_KEY environment varibles.
# These environment variables are set in the docker-compose.yml, and the service account used by PySpark
#> to read from and write to Minio are created by the minio-init container defined in docker-compose.yml

environ{'PATH': '/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin',
        'HOSTNAME': '3e1efd605e0e',
        'S3_ENDPOINT': 'http://minio:9000',
        'S3_BUCKET': 'test',
        'S3_ACCESS_KEY': 'sparkaccesskey',
        'S3_SECRET_KEY': 'sparksupersecretkey',
        'LANG': 'C.UTF-8',
        'GPG_KEY': 'A035C8C19219BA821ECEA86B64E628F8D684696D',
        'PYTHON_VERSION': '3.10.5',
        'PYTHON_PIP_VERSION': '22.0.4',
        'PYTHON_SETUPTOOLS_VERSION': '58.1.0',
        'PYTHON_GET_PIP_URL': 'https://github.com/pypa/get-pip/raw/6ce3639da143c5d79b44f94b04080abf2531fd6e/public/get-pip.py',
        'PYTHON_GET_PIP_SHA256': 'ba3ab8267d91fd41c58dbce08f76db99f747f716d85ce1865813842bb035524d',
        'HOME': '/root',
        'JPY_PARENT_PID': '1',
        'TERM': 'xterm-color',
        'CLICOLOR': '1',
        'PAGER': 'cat',
        'GIT_PAGER': 'cat',
        'MPLBACKEND': 'module://matplotlib_inline.backend_inline'}

In [3]:
S3_ACCESS_KEY = os.environ.get("S3_ACCESS_KEY")
S3_BUCKET = os.environ.get("S3_BUCKET")
S3_SECRET_KEY = os.environ.get("S3_SECRET_KEY")
S3_ENDPOINT = os.environ.get("S3_ENDPOINT")
# S3_ACCESS_KEY = "sparkaccesskey"
# S3_BUCKET = "test"
# S3_SECRET_KEY = "sparksupersecretkey"
# S3_ENDPOINT = "http://minio:9000"

## Configure Pyspark to Connect to Minio and Enable Delta-Lake Format

In [4]:
# This cell may take some time to run the first time, as it must download the necessary spark jars
conf = pyspark.SparkConf().setMaster("spark://spark:7077")
conf.set("spark.jars.packages", 'org.apache.hadoop:hadoop-aws:3.3.1,io.delta:delta-core_2.12:2.1.0')
# conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider')
conf.set('spark.hadoop.fs.s3a.endpoint', S3_ENDPOINT)
conf.set('spark.hadoop.fs.s3a.access.key', S3_ACCESS_KEY)
conf.set('spark.hadoop.fs.s3a.secret.key', S3_SECRET_KEY)
conf.set('spark.hadoop.fs.s3a.path.style.access', "true")
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

sc = pyspark.SparkContext(conf=conf)

# sc.setLogLevel("INFO")

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e9e2b5d0-550b-4fa3-955d-5fc310393b17;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.1/hadoop-aws-3.3.1.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.3.1!hadoop-aws.jar (260ms)
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.11.901/aws-java-sdk-bundle-1.11.901.ja

22/09/14 02:57:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
spark = pyspark.sql.SparkSession(sc)

## Read in Sample CSV Data from Local Filesystem

In [6]:
df = spark.read.option("header", "true").csv("/data/appl_stock.csv")

In [7]:
df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [8]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



## Modify Column Types

In [9]:
for col in ["Open", "High", "Low", "Close", "Adj Close"]:
    df = df.withColumn(col,df[col].cast('double'))
for col in ["Volume"]:
    df = df.withColumn(col, df[col].cast('int'))

In [10]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



## Write CSV Directly to Minio (Not as a Delta Table)

In [11]:
df.write.csv(f"s3a://{S3_BUCKET}/appl_stock.csv", mode="overwrite")

22/09/14 02:57:40 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/09/14 02:57:41 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


**Navigate to http://localhost:9090 and login to the Minio Console to see the CSV file**

(username and password for minio can be found in the environment variables section of the minio service definition in the docker-compose.yml)

# Write a Delta Lake Table in Minio using Spark

In [12]:
# Have to replace spaces in column names with underscores for Delta
delta_df = df
for col in delta_df.columns:
    delta_df = delta_df.withColumnRenamed(col, col.replace(" ","_"))

In [13]:
delta_df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj_Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [14]:
delta_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj_Close: double (nullable = true)



In [15]:
delta_table_name = "appl_stock_delta_table"

In [16]:
delta_df.write.format("delta").save(f"s3a://{S3_BUCKET}/{delta_table_name}", mode="overwrite")

**Navigate to http://localhost:9090 and login to the Minio Console to see the Delta Lake Table**

**Note that the Delta Lake Table includes both the data partitions and the metadata log**

(username and password for minio can be found in the environment variables section of the minio service definition in the docker-compose.yml)

# Read the Delta Table Back into Spark

In [17]:
new_delta_df = spark.read.format("delta").load(f"s3a://{S3_BUCKET}/{delta_table_name}")

In [18]:
new_delta_df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj_Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

## Delete Data From Delta Table

In [19]:
from delta.tables import *

In [20]:
delta_table = DeltaTable.forPath(spark, f"s3a://{S3_BUCKET}/{delta_table_name}")

In [21]:
delta_table.delete("Date < '2010-02-01'")

In [22]:
# delta_table.vacuum()

# .vacuum() is not really necessary for this example. For more info, see https://docs.delta.io/latest/delta-utility.html#remove-files-no-longer-referenced-by-a-delta-table

In [23]:
updated_df = delta_table.toDF()

In [24]:
updated_df.describe().show()
# Notice the min date due to the delete above

+-------+----------+------------------+------------------+------------------+------------------+-------------------+------------------+
|summary|      Date|              Open|              High|               Low|             Close|             Volume|         Adj_Close|
+-------+----------+------------------+------------------+------------------+------------------+-------------------+------------------+
|  count|      1743|              1743|              1743|              1743|              1743|               1743|              1743|
|   mean|      null|314.20635701549037| 317.0524208313251|310.96767053872645| 314.0739527596098|9.307720510613884E7| 75.52596069420551|
| stddev|      null|185.98853126511312|187.59247321328294| 184.0533368704578|185.82494613123126| 5.85169404895423E7| 28.28297910995164|
|    min|2010-02-01|              90.0|         90.699997|         89.470001|         90.279999|           11475900|         24.881912|
|    max|2016-12-30|        702.409988|        7

## Use Time Travel to Read a Previous Version of the Delta Table

In [25]:
previous_df = spark.read.format("delta").option("versionAsOf", 0).load(f"s3a://{S3_BUCKET}/{delta_table_name}")
previous_df.describe().show()
# Notice the min date, showing that we are reading from a previous version

+-------+----------+------------------+------------------+------------------+-----------------+-------------------+------------------+
|summary|      Date|              Open|              High|               Low|            Close|             Volume|         Adj_Close|
+-------+----------+------------------+------------------+------------------+-----------------+-------------------+------------------+
|  count|      1762|              1762|              1762|              1762|             1762|               1762|              1762|
|   mean|      null| 313.0763111589103| 315.9112880164581| 309.8282405079457|312.9270656379113|9.422577587968218E7| 75.00174115607275|
| stddev|      null|185.29946803981522|186.89817686485767|183.38391664371008|185.1471036170943|6.020518776592709E7| 28.57492972179906|
|    min|2010-01-04|              90.0|         90.699997|         89.470001|        90.279999|           11475900|         24.881912|
|    max|2016-12-30|        702.409988|        705.0700

In [26]:
delta_table.history().show()

+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      1|2022-09-14 02:57:58|  null|    null|   DELETE|{predicate -> ["(...|null|    null|     null|          0|  Serializable|        false|{numRemovedFiles ...|        null|Apache-Spark/3.3....|
|      0|2022-09-14 02:57:48|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|       null|  Serializable|        false|{numFiles -> 1, n...|        null|Apache-Spark/3.3....|
+-------+------

# Trigger Trino to Automatically Infer Schema from Delta Table and Make Data Available for End User Querying / Dashboarding

In [27]:
import requests
import json
from time import sleep

In [28]:
delta_table_name = "appl_stock_delta_table"
delta_schema_name = "my_schema"

In [29]:
def execute_trino_query(query, statement_endpoint = "http://trino:8080/v1/statement", user = "admin", password = ""):
    
    print(f"Executing query:\n{query}")
    res = requests.post(statement_endpoint,data = query.encode("UTF8"), auth=requests.auth.HTTPBasicAuth(user,password))
    
    data = []
    cols = None
    while True:
        json_res = res.json()
        state = json_res.get("stats").get("state")
        print(f"State: {state}")

        res_data = json_res.get("data")
        if res_data:
            data.extend(res_data)
        
        res_cols = json_res.get("columns")
        if res_cols:
            cols = [i["name"] for i in res_cols]
            
        next_uri = json_res.get("nextUri")
        if next_uri:
            sleep(.5)
            res = requests.get(next_uri)
        else:
            return [dict(zip(cols, d)) for d in data]
                
            


## Trigger Trino to Read Delta Table Schema

In [30]:
create_schema_statement = f"""
CREATE SCHEMA IF NOT EXISTS delta.my_schema
WITH (location = 's3a://{S3_BUCKET}/')
"""

create_table_statement = f"""CREATE TABLE IF NOT EXISTS delta.{delta_schema_name}.{delta_table_name} (
  dummy bigint
)
WITH (
  location = 's3a://{S3_BUCKET}/{delta_table_name}'
)
"""

In [31]:
for query in [create_schema_statement, create_table_statement]:
    print(execute_trino_query(query))

Executing query:

CREATE SCHEMA IF NOT EXISTS delta.my_schema
WITH (location = 's3a://test/')

State: QUEUED
State: QUEUED
State: QUEUED
State: QUEUED
State: FINISHED
[{'result': True}]
Executing query:
CREATE TABLE IF NOT EXISTS delta.my_schema.appl_stock_delta_table (
  dummy bigint
)
WITH (
  location = 's3a://test/appl_stock_delta_table'
)

State: QUEUED
State: QUEUED
State: QUEUED
State: FINISHED
[{'result': True}]


## Query Data from Table 

In [32]:
LIMIT = 10
select_statement = f"SELECT * FROM delta.{delta_schema_name}.{delta_table_name}"
if LIMIT and type(LIMIT) == int:
    select_statement += f" LIMIT {LIMIT}"

In [33]:
data = execute_trino_query(select_statement)

Executing query:
SELECT * FROM delta.my_schema.appl_stock_delta_table LIMIT 10
State: QUEUED
State: QUEUED
State: QUEUED
State: RUNNING
State: FINISHING
State: FINISHED


In [34]:
print(data)

[{'date': '2010-02-01', 'open': 192.36999699999998, 'high': 196.0, 'low': 191.29999899999999, 'close': 194.729998, 'volume': 187469100, 'adj_close': 25.229131}, {'date': '2010-02-04', 'open': 196.730003, 'high': 198.370001, 'low': 191.570005, 'close': 192.050003, 'volume': 189413000, 'adj_close': 24.881912}, {'date': '2010-02-05', 'open': 192.63000300000002, 'high': 196.0, 'low': 190.850002, 'close': 195.460001, 'volume': 212576700, 'adj_close': 25.323710000000002}, {'date': '2010-02-08', 'open': 195.690006, 'high': 197.88000300000002, 'low': 193.999994, 'close': 194.11999699999998, 'volume': 119567700, 'adj_close': 25.1501}, {'date': '2010-02-09', 'open': 196.419996, 'high': 197.499994, 'low': 194.749998, 'close': 196.19000400000002, 'volume': 158221700, 'adj_close': 25.418289}, {'date': '2010-02-02', 'open': 195.909998, 'high': 196.319994, 'low': 193.37999299999998, 'close': 195.859997, 'volume': 174585600, 'adj_close': 25.375532999999997}, {'date': '2010-02-03', 'open': 195.169994, 